In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install peft --no-index --find-links=/kaggle/input/peft-pkg/peft_pkg

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, TaskType, PeftModel
from tqdm import tqdm
from collections import Counter

In [ ]:
def data_transfer(dataframe, instruction):
    x_cols = ['prompt', 'response_a', 'response_b', 'id']
    x = dataframe[x_cols]

    messages = []
    id_list = []
    for idx in range(len(x)):
        x_line = x.iloc[idx,:]

        prompt = 'PROMPT: ' + x_line['prompt']
        model_a = 'MODEL_A: ' + x_line['response_a']
        model_b = 'MODEL_B: ' + x_line['response_b']
        text = prompt + model_a + model_b

        message = {
            'instruction': instruction,
            'input': text,
            'output': ""
        }

        messages.append(message)
        id_list.append(x_line['id'])
    
    return messages, id_list

In [ ]:
def predict(msg, model, tokenizer, repeat=1):
    device = "cuda"
    
    model.to(device)
    messages = [
        {"role": "system", "content": msg['instruction']},
        {"role": "user", "content": msg['input']}
    ]

    repeat_log = []
    for _ in range(repeat):
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
        generated_ids = model.generate(
            model_inputs.input_ids,
            max_new_tokens=5
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
    
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        repeat_log.append(response)
    response = Counter(repeat_log).most_common(1)[0][0]

    return response

In [ ]:
model_pt = "/kaggle/input/qwen_0.5b_instruct/transformers/default/1"
lora_pt = "/kaggle/input/qwen_0.5b_lorafinetune_5ktoken_62kckpt/transformers/default/1"

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

def load_models(qwen_dir, ckpt):
    tokenizer = AutoTokenizer.from_pretrained(qwen_dir)
    model = AutoModelForCausalLM.from_pretrained(qwen_dir)
    model = PeftModel.from_pretrained(model, model_id=ckpt, config=config)
    model.eval()

    return tokenizer, model

tokenizer, model = load_models(model_pt, lora_pt)

In [ ]:
test_dir = "/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet"
test = pd.read_parquet(test_dir)

In [ ]:
instruction = """In the text provided below, PROMPT is the question presented; MODEL_A is the response from the first model; MODEL_B is the response from the second model. Please select the best answer from the two responses above. If the first answer is better, return "model_a"; if the second answer is better, return "model_b"."""
test, id_list = data_transfer(test, instruction)

In [ ]:
predictions = []
for msg in tqdm(test):
    try:
        response = predict(msg, model, tokenizer, 5)
        predictions.append(response)
    except:
        predictions.append("model_a")

In [ ]:
sub = pd.DataFrame({
    'id': id_list,
    'winner': predictions
})

In [ ]:
sub.to_csv("submission.csv",index=False)

In [ ]:
sub